In [1]:
import pandas as pd 
import numpy as np
# import datetime
import logging
from tabulate import tabulate
import sys

In [3]:
file = "/home/amr/Desktop/Downloads_From_Internet/26-01-2024-18_35_47.xlsx"
# final = "40 SAR"
# file = sys.argv[1]
# final = sys.argv[2]

In [4]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

Formatter = logging.Formatter('%(message)s')

FileHandler = logging.FileHandler(f"{file.rsplit('/',1)[0]}/out.log")
FileHandler.setFormatter(Formatter)

logger.addHandler(FileHandler)



In [5]:
pd.set_option('display.colheader_justify','center')
pd.set_option('display.width', 400)
pd.set_option('display.expand_frame_repr', False)


In [6]:
data = pd.read_excel(file)
data['Date'] = pd.to_datetime(data['Date']).dt.date
# data = data[data['Date'] >= datetime.date(year=2023,month=11, day=1)]

In [7]:
f_balance = round(float(final.split()[0].replace(',','')),3)
f_balance

40.0

In [8]:
idx_dic = {
    'Free Credit': '1',
    'Cash Out': '2',
    'Balance Adjustment': '3',
    'Compensation': '4',
    'Cash Collection Cycle': '5',
    'Bosta Fees Cycle': '6',
 }

In [11]:
data['Category'].replace(idx_dic,inplace=True)
datax = data.sort_values(by=['Date','Category','Transactions ID'])
datax.reset_index(drop=True,inplace=True)
swapped_dict = {value: key for key, value in idx_dic.items()}
datax.replace(swapped_dict,inplace=True)

In [12]:
# datax['Balance'], datax['Amount'] = datax['Balance'], datax['Amount']

balance = datax['Balance'].round(3).to_list()
amount = datax['Amount'].round(3).to_list()
lookout = round(f_balance - balance[-1],3)
lookout

-6813.0

In [13]:
%%capture
diff = 0
# date_IDs = []
for a,b,c,d in zip(amount[1:],balance[:-1],balance[1:],datax['Transactions ID'].to_list()[1:]):
    sum = round(a + b,3)
    c = round(c,3)
    if sum == c:
        continue
    else:
        entry = datax[datax['Transactions ID'] == d]
        index = entry.index[0]
        out = entry.copy()
        if index - 1 >= 0:
            out = pd.concat([out,datax.iloc[index - 1].to_frame().T])# , ignore_index = True)
        if index + 1 < len(datax):
            out = pd.concat([datax.iloc[index + 1].to_frame().T, out])# , ignore_index = True)


        
        diff += round(sum - c,3)
        # if diff == lookout:
        # logger.info(entry['Transactions ID'].values[0])
        logger.info(f"\n\nCorrect sum = {sum}\t\tDifference = {round(sum - c,3)}\t\t Cumalitive Difference = {diff}")
        logger.info(f"{a},{b},{sum},{c}")
        logger.info(tabulate(out, headers='keys', tablefmt='fancy_grid',colalign=tuple(["center"]*entry.shape[1]),showindex='never'))
        # date_IDs.append((entry['Transactions ID'].values[0], str(entry['Date'].values[0]), diff))
        
        

In [ ]:
# f_balance == round(diff + balance[-1],3)
# diff, f_balance - balance[-1]